In [1]:
# Disable Cache
import os, sys
sys.dont_write_bytecode = True
os.environ["NUMBA_DISABLE_JIT"] = "1"

import quantfreedom as qf
import pandas as pd
from quantfreedom.backtester.nb.simulate import simulate_from_signals
from quantfreedom.backtester.base.base import rsi_below_entries
from quantfreedom.backtester.enums.enums import (
    LeverageMode, 
    SizeType,
    OrderType,
    SL_BE_or_Trail_BasedOn,
    )
# qf.clear_cache()

prices = pd.read_csv('30min.csv', index_col='time')
entries = rsi_below_entries(
    timeperiods=[20],
    below_ranges=[50],
    prices=prices.close
)

In [2]:
df, order_rec, cart = simulate_from_signals(
    open_prices=prices.open.values,
    high_prices=prices.high.values,
    low_prices=prices.low.values,
    close_prices=prices.close.values,
    entries=entries.values,
    equity=1000.,
    fee_pct=.06,
    mmr=.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=3.,
    risk_rewards=[5],
    size_pct=1.,
    tsl_pcts=[2],
    # gains_pct_filter=100,
    sl_to_be=True,
    tsl_true_or_false=True,
    # tsl_based_on=SL_BE_or_Trail_BasedOn.low_price,
    # tsl_trail_by_pct=1,
    # tsl_when_pct_from_avg_entry=1,
)
pd.DataFrame(order_rec).head(10)

ValueError: You can't have sl_to_be and tsl_true_or_false both be true

In [ ]:
pd.DataFrame(df)

In [ ]:
pd.DataFrame(cart).T